In [ ]:
"""Performs face alignment and calculates L2 distance between the embeddings of images."""

# MIT License
# 
# Copyright (c) 2016 David Sandberg
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from tensorflow.python.platform import gfile
from scipy import misc
import tensorflow as tf
import numpy as np
import sys
import os
import argparse
import facenet
import align.detect_face
import time as tm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pandas  import Series
import pylab

class Faster_style(object):
    
    def __init__(self): 
            self.sess =tf.Session()
            #Faster_style.load_model('20170511-185253.pb')
            self.load_model('20170512-110547.pb')
            self.images_placeholder =  tf.get_default_graph().get_tensor_by_name("input:0")
            self.embeddings =  tf.get_default_graph().get_tensor_by_name("embeddings:0")
            self.phase_train_placeholder =  tf.get_default_graph().get_tensor_by_name("phase_train:0")
           
     
    def load_model(self,model):
        # Check if the model is a model directory (containing a metagraph and a checkpoint file)
        #  or if it is a protobuf file with a frozen graph
        model_exp = os.path.expanduser(model)
        if (os.path.isfile(model_exp)):
            print('Model filename: %s' % model_exp)
            with gfile.FastGFile(model_exp,'rb') as f:
                graph_def = tf.GraphDef()
                graph_def.ParseFromString(f.read())
                tf.import_graph_def(graph_def, name='')
               
        else:
            print('Model directory: %s' % model_exp)
            meta_file, ckpt_file = get_model_filenames(model_exp)
            
            print('Metagraph file: %s' % meta_file)
            print('Checkpoint file: %s' % ckpt_file)
          
            saver = tf.train.import_meta_graph(os.path.join(model_exp, meta_file))
            saver.restore(self.sess, os.path.join(model_exp, ckpt_file))
          
        
        
    def main(self,args,image_files):
        t1 = tm.time()
        print('main')
        
        images,img_files = self.load_and_align_data(image_files, args.image_size, args.margin, args.gpu_memory_fraction) 
        
        feed_dict = {self.images_placeholder: images,  self.phase_train_placeholder:False }
        emb =self.sess.run(self.embeddings, feed_dict=feed_dict)    
        print(emb.shape)
       # nrof_images = len(args.image_files)
        t2 = tm.time()
        print ("==========total take {0}".format(t2 - t1))
        print('Images:')
        return emb,img_files
       # for i in range(nrof_images):
       #     print('%1d: %s' % (i, args.image_files[i]))
        #     # Print distance matrix
        #    print('Distance matrix')
        #    print('    ', end='')
        print('')
        #for i in range(nrof_images):
         #    print('    %1d     ' % i, end='')
       # print('')
       # for i in range(nrof_images):
        #    print('%1d  ' % i, end='')
        #    for j in range(nrof_images):
          #      dist = np.sqrt(np.sum(np.square(np.subtract(emb[i,:], emb[j,:]))))
         #       print('  %1.4f  ' % dist, end='')
          #  print('')
                      
    def load_and_align_data(self,image_paths, image_size, margin, gpu_memory_fraction):   
        minsize = 20 # minimum size of face  
        threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
        factor = 0.709 # scale factor
        print('detect and align data')
        with tf.Graph().as_default():
            gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
            sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
            with sess.as_default():
                pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)
      
        nrof_samples = len(image_paths)
        img_files=[]
        img_list = []
        for i in range(nrof_samples):
            img = misc.imread(os.path.expanduser(image_paths[i]), mode='RGB')
            img_size = np.asarray(img.shape)[0:2]
            bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
            if(bounding_boxes.shape[0]>0):
                det = np.squeeze(bounding_boxes[0,0:4])
                bb = np.zeros(4, dtype=np.int32)
                bb[0] = np.maximum(det[0]-margin/2, 0)
                bb[1] = np.maximum(det[1]-margin/2, 0)
                bb[2] = np.minimum(det[2]+margin/2, img_size[1])
                bb[3] = np.minimum(det[3]+margin/2, img_size[0])
                cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
                aligned = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
                prewhitened = facenet.prewhiten(aligned)
                img_list.append(prewhitened)
                img_files.append(image_paths[i])        
        images = np.stack(img_list)
        return images,img_files

    def parse_arguments(self,argv):
        parser = argparse.ArgumentParser()
   
        parser.add_argument('model', type=str, 
            help='Could be either a directory containing the meta_file and ckpt_file or a model protobuf (.pb) file')
        parser.add_argument('image_files', type=str, nargs='+', help='Images to compare')
        parser.add_argument('--image_size', type=int,
            help='Image size (height, width) in pixels.', default=160)
        parser.add_argument('--margin', type=int,
            help='Margin for the crop around the bounding box (height, width) in pixels.', default=44)
        parser.add_argument('--gpu_memory_fraction', type=float,
            help='Upper bound on the amount of GPU memory that will b e used by wthe process.', default=1.0)
        return parser.parse_args(argv)

#f __name__ == '__main__':
 #   sys.argv=['compare.py','20170511-185253.pb','image1.jpg','image2.jpg','image3.jpg','image4.jpg','image5.jpg']
 #   main(parse_arguments(sys.argv[1:]))
    def generate_csv(self): 
        style=Faster_style()
        sys.argv=['compare.py','20170511-185253.pb','image.jpg']
        argv=style.parse_arguments(sys.argv[1:])
        image_files=[]
        files = os.listdir('files')
        for i in range(len(files)):
            image_name = files[i]
        
            facedir = os.path.join('files', image_name)
           
            image_files.append(facedir)
        count=len(image_files)/100
        print(count)
        for i in range(int(count)):
            imgs=image_files[i*100:i*100+100]
            emb,emb_img_files=style.main(argv,imgs)
            print(emb.shape)

            emb_df=pd.DataFrame(emb)
            emb_img_files=np.reshape(emb_img_files,(-1,1))
            print(emb_img_files.shape)
            path_df=pd.DataFrame(emb_img_files)

            res=pd.concat([emb_df,path_df],axis=1, ignore_index=True)
            data_info=pd.read_csv('data1.csv',header=None)
            data_info=data_info.append(res)
            data_info.to_csv('data1.csv',index=False,header=None)
           
        

    
    def search(self,image_path):  
       
        sys.argv=['compare.py','20170511-185253.pb','image.jpg']
        argv=style.parse_arguments(sys.argv[1:])
        image_files=[]
        image_files.append(image_path)
        emb,_=self.main(argv,image_files)
        data_info2=pd.read_csv('data1.csv')
        columns= data_info2.iloc[0:,0:128]
        source_imgs=data_info2.iloc[0:,128:129]
        source_imgs=np.array(source_imgs)
        rows=columns.shape[0]
        marix=np.mat(columns)
        distlist=[]
        print(marix.shape)
        for i in range(rows):
            dist = np.sqrt(np.sum(np.square(np.subtract(marix[i,:],emb[0,:]))))
            distlist.append(dist)   
        b = np.sort(distlist, axis=0)
        b1 = np.argsort(distlist)
        likesImg=source_imgs[b1]
        for i in range(10):
            filepath=likesImg[i,0]
            lena = mpimg.imread(filepath) 
            lena.shape #(512, 512, 3)
            print(lena.shape)
            plt.imshow(lena) # 显示图片
            pylab.show()  
        
 

style=Faster_style()
#style.generate_csv()

style.search('image.jpg')

Model filename: 20170512-110547.pb
main
detect and align data


In [12]:
import compare
import sys

sys.argv=['compare.py','20170511-185253.pb','image.jpg','image1.jpg']
args=compare.parse_arguments(sys.argv[1:])

for i in range(100):
    compare.main(args)


Creating networks and loading parameters
Model filename: 20170511-185253.pb


KeyboardInterrupt: 